In [ ]:
from fastai.vision.all import load_learner

learner = load_learner("export.pkl")

Plot 10 Random images for each category and show results

In [ ]:
# 3x10 Grid Evaluation - 10 random images from each class
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path
import random

# Setup
classes = learner.dls.vocab
valid_path = Path("data/valid")

# Set random seed for reproducible results (optional)
random.seed(42)

NUM_IMAGES = 10
fig, axes = plt.subplots(len(classes), NUM_IMAGES, figsize=(30, 15))

# Process each class (row)
for row, class_name in enumerate(classes):
    class_dir = valid_path / class_name

    # Get all available images and randomly select 10
    all_images = list(class_dir.glob("*.png"))
    selected_images = random.sample(all_images, min(NUM_IMAGES, len(all_images)))

    # Pad with None if we don't have enough images
    while len(selected_images) < NUM_IMAGES:
        selected_images.append(None)

    # Process each selected image
    for col in range(NUM_IMAGES):
        img_path = selected_images[col]

        if img_path and img_path.exists():
            pred_class, pred_idx, probs = learner.predict(img_path)

            # Load and display image
            img = Image.open(img_path)
            axes[row, col].imshow(img)
            axes[row, col].axis("off")

            # Check if prediction is correct
            is_correct = str(pred_class) == class_name
            confidence = probs.max().item()

            # Color based on correctness
            color = "green" if is_correct else "red"

            # Create probability bars text above the image
            image_title = f"{img_path.name}\nPred: {pred_class}\n"
            for cls, prob in zip(learner.dls.vocab, probs):
                percentage = prob.item() * 100
                bar_length = int(percentage / 5)  # Scale to fit
                bar = "█" * bar_length + "░" * (20 - bar_length)
                # Pad class name to 10 characters for alignment
                cls_padded = f"{cls}:".ljust(10)
                image_title += f"{cls_padded} {percentage:4.1f}% {bar}\n"

            # Add the probability bars as title above the image
            axes[row, col].set_title(
                image_title.strip(),
                fontsize=7,
                color=color,
                fontweight="bold",
                fontfamily="monospace",
            )

        else:
            # Handle missing image
            axes[row, col].text(
                0.5,
                0.5,
                "No image\navailable",
                ha="center",
                va="center",
                transform=axes[row, col].transAxes,
                fontsize=10,
                color="gray",
            )
            axes[row, col].axis("off")

# Add row labels
for row, class_name in enumerate(classes):
    fig.text(
        0.02,
        0.75 - row * 0.3,
        class_name.upper() + "_GT",
        fontsize=14,
        fontweight="bold",
        rotation=90,
        va="center",
    )

plt.tight_layout()
plt.subplots_adjust(left=0.05, top=0.85)  # More space for probability bars above images
plt.show()

Load learner

In [ ]:
from fastai.vision.all import load_learner

learner = load_learner("export.pkl")

In [ ]:
# Pretty evaluation with image display
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path

# Test images
test_images = [
    ("data/valid/linear/18.png", "linear"),
    ("data/valid/cubic/18.png", "cubic"),
    ("data/valid/quadratic/18.png", "quadratic"),
]

# Create subplot for all 3 images
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle("Graph Classification Results", fontsize=16, fontweight="bold")

for i, (img_path, true_label) in enumerate(test_images):
    # Make prediction
    pred_class, pred_idx, probs = learner.predict(img_path)

    # Load and display image
    img = Image.open(img_path)
    axes[i].imshow(img)
    axes[i].axis("off")

    # Check if correct
    is_correct = str(pred_class) == true_label
    confidence = probs.max().item()

    # Color title based on correctness
    color = "green" if is_correct else "red"

    # Set image title
    title = f"True: {true_label}\nPred: {pred_class}\nConf: {confidence:.1%}"
    axes[i].set_title(title, fontsize=12, color=color, fontweight="bold")

    # Print detailed results
    print(f"\n{'=' * 40}")
    print(f"📸 IMAGE: {Path(img_path).name}")
    print(f"{'=' * 40}")
    print(f"True Label:    {true_label}")
    print(f"Predicted:     {pred_class}")
    print(f"Confidence:    {confidence:.1%}")
    print(f"Status:        {'✅ CORRECT' if is_correct else '❌ WRONG'}")
    print("\nProbability Breakdown:")
    print(f"{'Class':<12} {'Probability':<12} {'Bar'}")
    print(f"{'-' * 40}")

    for cls, prob in zip(learner.dls.vocab, probs):
        prob_val = prob.item()
        bar_length = int(prob_val * 30)
        bar = "█" * bar_length + "░" * (30 - bar_length)
        print(f"{cls:<12} {prob_val:>7.1%}       {bar}")

plt.tight_layout()
plt.show()